In [11]:
import os
os.chdir('/nfsmnt/qiujunwen/AmphionVALLEv2') # change to your amphion root path

In [12]:
# you can download pretrained checkpoints from 
# or use your own pretrained weights
ar_model_path = '/nfsmnt/qiujunwen/ckpt/valle_gpt_simple/mls/valle_ar_mls_encodec.bin'  # download the file on https://huggingface.co/jiaqili3/vallex/resolve/main/valle_ar_mls_encodec.bin?download=true
nar_model_path = '/nfsmnt/qiujunwen/ckpt/valle_gpt_simple/mls/valle_nar_mls_encodec.bin'  # download the file on https://huggingface.co/jiaqili3/vallex/resolve/main/valle_nar_mls_encodec.bin?download=true

In [13]:
from models.tts.valle_gpt_simple.valle_inference import ValleInference
# change to device='cuda' to use CUDA GPU for fast inference
# change "use_vocos" to True would give better sound quality
# If you meet problem with network, you could set "use_vocos=False", though would give bad quality
model = ValleInference(use_vocos=False, ar_path=ar_model_path, nar_path=nar_model_path, device='cpu')

In [14]:
# prepare inference data
import librosa
import torch
wav, _ = librosa.load('/nfsmnt/qiujunwen/ckpt/valle_gpt_simple/mls/example.wav', sr=24000)
wav = torch.tensor(wav, dtype=torch.float32)
import IPython
IPython.display.Audio('/nfsmnt/qiujunwen/ckpt/valle_gpt_simple/mls/example.wav')

In [15]:
prompt_transcript = 'His glance, at first wandering, fixed itself upon Morrel.'
target_transcript = 'It presents a unified framework that is inclusive of diverse generation tasks and models, with the added bonus of being easily extendable for new'
from utils.g2p.g2p import phonemizer_g2p as g2p
prompt_transcript = g2p(prompt_transcript, 'en')[1]
target_transcript = g2p(target_transcript, 'en')[1]


In [16]:
prompt_transcript = torch.tensor(prompt_transcript).long()
target_transcript = torch.tensor(target_transcript).long()
transcript = torch.cat([prompt_transcript, target_transcript], dim=-1)
batch = {
        'speech': wav.unsqueeze(0),
        'phone_ids': transcript.unsqueeze(0),
}

In [17]:
configs = [dict(
    top_p=1.0,
    top_k=10,
    temperature=1.1,
    repeat_penalty=1.15,
    max_length=2000,
    num_beams=1,
)] # model inference hyperparameters
output_wav = model(batch, configs)

In [18]:
output_wav

tensor([[[ 0.0004,  0.0012,  0.0015,  ..., -0.0005,  0.0013,  0.0005]]])

In [19]:
import torchaudio
torchaudio.save('out.wav', output_wav.squeeze(0), 24000)


In [20]:
import os
os.path.isfile('out.wav')
import IPython.display
IPython.display.Audio('out.wav')